In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/neccam/slt.git

%cd slt

!chmod +x data/download.sh
!data/download.sh

!cp ../drive/MyDrive/slt/requirements.txt requirements.txt
!pip -q install -r requirements.txt

Cloning into 'slt'...
remote: Enumerating objects: 99, done.
remote: Total 99 (delta 0), reused 0 (delta 0), pack-reused 99
Unpacking objects: 100% (99/99), done.
/content/slt
--2021-01-03 14:13:06--  http://cihancamgoz.com/files/cvpr2020/phoenix14t.pami0.train
Resolving cihancamgoz.com (cihancamgoz.com)... 192.232.218.145
Connecting to cihancamgoz.com (cihancamgoz.com)|192.232.218.145|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1935289362 (1.8G)
Saving to: ‘phoenix14t.pami0.train’

phoenix14t.pami0.tr 100%[===================>]   1.80G  20.4MB/s    in 94s     

2021-01-03 14:14:41 (19.5 MB/s) - ‘phoenix14t.pami0.train’ saved [1935289362/1935289362]

--2021-01-03 14:14:41--  http://cihancamgoz.com/files/cvpr2020/phoenix14t.pami0.dev
Resolving cihancamgoz.com (cihancamgoz.com)... 192.232.218.145
Connecting to cihancamgoz.com (cihancamgoz.com)|192.232.218.145|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 130831227 (125M)
Saving to: 

In [8]:
!pip -q install -r requirements.txt

     |████████████████████████████████| 753.4MB 21kB/s 
     |████████████████████████████████| 81kB 11.7MB/s 
     |████████████████████████████████| 4.0MB 64.3MB/s 
     |████████████████████████████████| 61kB 9.1MB/s 
     |████████████████████████████████| 133kB 59.9MB/s 
     |████████████████████████████████| 327kB 57.6MB/s 
ERROR: tensorflow 2.4.0 has requirement absl-py~=0.10, but you'll have absl-py 0.9.0 which is incompatible.
ERROR: tensorflow 2.4.0 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.4.0 has requirement google-pasta~=0.2, but you'll have google-pasta 0.1.8 which is incompatible.
ERROR: tensorflow 2.4.0 has requirement grpcio~=1.32.0, but you'll have grpcio 1.26.0 which is incompatible.
ERROR: tensorflow 2.4.0 has requirement keras-preprocessing~=1.1.2, but you'll have keras-preprocessing 1.1.0 which is incompatible.
ERROR: tensorflow 2.4.0 has requirement numpy~=1.19.2, but you'll have numpy 1.18.1 which is inco

In [3]:
!mkdir data/PHOENIX2014T 

!mv phoenix14t.pami0.dev data/PHOENIX2014T 
!mv phoenix14t.pami0.test data/PHOENIX2014T 
!mv phoenix14t.pami0.train data/PHOENIX2014T

!unzip -q '../drive/MyDrive/slt/keypoints/train_body_pose3d.zip' -d data
!unzip -q '../drive/MyDrive/slt/keypoints/test_keypoints.zip' -d data
!unzip -q '../drive/MyDrive/slt/keypoints/dev_keypoints.zip' -d data

In [4]:
from six.moves import cPickle
import gzip
import torch
import os
import numpy as np
from tqdm import tqdm
import pickle

wd='data'

def load_dataset_file(filename):
    with gzip.open(filename, "rb") as f:
        loaded_object = pickle.load(f)
        return loaded_object

body_parts = ['body_pose3d']

for s in ['dev', 'test', 'train']:
  filename = "data/PHOENIX2014T/phoenix14t.pami0.{}".format(s)
  src_features = load_dataset_file(filename)
  dst_filename = "data/PHOENIX2014T/phoenix14t.pami1.{}".format(s)
  for src in tqdm(src_features):
    name = os.path.basename(src['name']) + '.npy'
    
    new_features_list = []

    for body_part in body_parts:
      dst = os.path.join(os.path.join(wd, s), os.path.join(body_part, name))

      new_features = np.load(dst)
      new_features = new_features.reshape((new_features.shape[0], np.prod(new_features.shape[1: ])))
      new_features_list.append(new_features)
          
    new_features = np.concatenate(new_features_list, axis=1)
    src['sign'] = torch.Tensor(new_features)

  with gzip.open(dst_filename, "wb") as f:
    pickle.dump(src_features, f)

100%|██████████| 7096/7096 [00:03<00:00, 2175.15it/s]


In [5]:
src['sign'].shape

torch.Size([43, 39])

In [6]:
import yaml
with open("configs/sign.yaml") as f:
     list_doc = yaml.load(f)

for s in ['train', 'test', 'dev']:
  list_doc['data'][s] = ['PHOENIX2014T/phoenix14t.pami{}.{}'.format(i, s) for i in range(2)]

list_doc['data']['feature_size'] = [1024, 39]
list_doc['training']['recognition_loss_weight'] = 20

with open("configs/sign.yaml", "w") as f:
    yaml.dump(list_doc, f)

In [9]:
!python -m signjoey train configs/sign.yaml

Streaming output truncated to the last 5000 lines.
2021-01-03 14:34:04,111 	Gloss Hypothesis:	DABEI **** REGION MILD
2021-01-03 14:34:04,111 	Gloss Alignment :	      D    S          
2021-01-03 14:34:04,111 	--------------------------------------------------------------------------------------------------------------------
2021-01-03 14:34:04,112 	Text Reference  :	deutlich milder ist  es unter den  wolken norddeutschlands .
2021-01-03 14:34:04,112 	Text Hypothesis :	******** dabei  wird es auch  noch etwas  milder           .
2021-01-03 14:34:04,112 	Text Alignment  :	D        S      S       S     S    S      S                 
2021-01-03 14:34:04,112 ========================================================================================================================
2021-01-03 14:34:04,112 Logging Sequence: dev/04January_2010_Monday_heute-7401
2021-01-03 14:34:04,113 	Gloss Reference :	REGION ANDERS MILD HINREICHEND  HEUTE NACHT HINREICHEND SCHNEE
2021-01-03 14:34:04,113 	Gloss Hy